# Instructions:

Normalize the following block of data by copying the data below, except the triple quotes, and placing the data in a file called employee_data.csv

1. Import the file using core python

2. Normalize the data and separate into component tables

3. Use SQL to create the tables and populate them

4. Answer the following questions using SQL:
    - Provide a count of the employees by gender
    - Provide a count of the employees by dept name
    - Provide a list of employees in last name, first name order
    - Provide a list of employees in tenure order (active employees only) in descending order
    - Provide a list of employees by manager name in alphabetical order by last name
    - Provide a list of employees by region
    - Provide a list of employees who have exceeded their allotment of sick days

# Opening CSV and Building the Database:

## Import all the tools

In [1]:
import csv
import sqlite3
from datetime import date,timedelta

## Open and read the CSV 

In [2]:
with open('employee_data.csv', 'r', newline='') as file:
    data = [data_set for data_set in csv.reader(file, delimiter=',', quotechar="'")]
    data_types = data.pop(0)

## Normalizing data into a dictionary of tables

In [3]:

service_dates = {
    data_set[0]: {
        'id': data_set[0],
        'hire_date': data_set[4] if data_set[4] != '' else '',
        'term_date': data_set[10] if data_set[10] != '' else '',
        'years_of_service': (
                date(
                    int(data_set[10][:4]),
                    int(data_set[10][5:7]),
                    int(data_set[10][-2:])
                )
                -
                date(
                    int(data_set[4][:4]),
                    int(data_set[4][5:7]),
                    int(data_set[4][-2:])
                )
            )
            // timedelta(days=365)
            if data_set[10] else(
                date.today() - date(
                    int(data_set[4][:4]),
                    int(data_set[4][5:7]),
                    int(data_set[4][-2:])
            )
        )
        // timedelta(days=365)
    }
    for data_set in data
}
departments = {
    data_set[6]: data_set[7]
    for data_set in data
}
sick_days = {
    data_set[0]: {
        'id': data_set[0],
        1: data_set[-3] if data_set[-3] != '' else '',
        2: data_set[-2] if data_set[-2] != '' else '',
        3: data_set[-1] if data_set[-1] != '' else ''
    }
    for data_set in data
}
regions = {
    'NORTH': 'N',
    'NORTH EAST': 'NE',
    'NORTH WEST': 'NW',
    'SOUTH': 'S',
    'SOUTH EAST': 'SE',
    'SOUTH WEST': 'SW',
    'EAST': 'E',
    'WEST': 'W'
}
genders = {
    'Male': 'M',
    'Female': 'F'
}
marital_codes = {
    'Married':'M',
    'Divorced': 'D',
    'Single': 'S',
    'Widowed': 'W' 
}
exemptions = {
    'NON-EXEMPT': 0,
    'EXEMPT': 1
}
employees = {
    data_set[0]: {
        'id': data_set[0],
        'last_name': data_set[2],
        'first_name': data_set[3],
        'gender_code': genders[data_set[5]],
        'marital_code': marital_codes[data_set[1]],
        'exemption': exemptions[data_set[9]],
        'manager_id': data_set[8],
        'dept_code': data_set[6],
        'region_code': regions[data_set[11]],
        'accrued_sick_days': int(data_set[12]),
        'available_sick_days': int(data_set[12]) - len(sick_days[data_set[0]])
    }
    for data_set in data
}



## Create the databse

In [4]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Create the tables
cursor.executescript('''
    create table if not exists employees (
        employee_id text,
        last_name text,
        first_name text,
        gender_code text,
        marital_code text,
        exemption_code int,
        manager_id text,
        dept_code text,
        region_code text,
        accrued_sick_days int,
        available_sick_days int
    );
    create table if not exists genders (
        code text,
        description text
    );
    create table if not exists marital_codes (
        code text,
        description text
    );
    create table if not exists exemptions (
        code text,
        description text
    );
    create table if not exists departments (
        code text,
        description text
    );
    create table if not exists service_dates (
        employee_id text,
        hire_date text,
        term_date text,
        years_of_service int
    );
    create table if not exists regions (
        code text,
        description text
    );
    create table if not exists sick_days (
        employee_id text,
        sick_day1 text,
        sick_day2 text,
        sick_day3 text
    );
    '''
)

# Commit and close connection
connection.commit()
connection.close()

## Insert data into tables

In [5]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Populate employees
for employee in employees.values(): 
    cursor.execute('''
    insert into employees values (?,?,?,?,?,?,?,?,?,?,?) 
    ''',
    tuple(employee.values())
    )
# Populate genders
for gender in genders.items(): 
    cursor.execute('''
    insert into genders values (?,?) 
    ''',
    gender[::-1]
    )
# Populate marital_codes
for marital_code in marital_codes.items(): 
    cursor.execute('''
    insert into marital_codes values (?,?) 
    ''',
    marital_code[::-1]
    )
# Populate exemptions
for exemption in exemptions.items(): 
    cursor.execute('''
    insert into exemptions values (?,?) 
    ''',
    exemption[::-1]
    )
# Populate departsments
for department in departments.items(): 
    cursor.execute('''
    insert into departments values (?,?) 
    ''',
    department
    )
# Populate service_dates
for service_date in service_dates.values(): 
    cursor.execute('''
    insert into service_dates values (?,?,?,?) 
    ''',
    tuple(service_date.values())
    )
# Populate regions
for region in regions.items(): 
    cursor.execute('''
    insert into regions values (?,?) 
    ''',
    region[::-1]
    )
# Populate sick_days
for sick_day in sick_days.values(): 
    cursor.execute('''
    insert into sick_days values (?,?,?,?) 
    ''',
    tuple(sick_day.values())
    )

# Commit and close connection
connection.commit()
connection.close()

# Querying Database:

## Provide a count of the employees by gender

In [6]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

 # Select data
cursor.execute('''
    select genders.description,count(gender_code) from employees
    left join genders on employees.gender_code=genders.code
    group by gender_code;
    '''
    )
# Fetch the query
genders = [f'{gender[0]}s: {gender[1]}\n' for gender in cursor.fetchall()]
print(f'Number of Employees by Gender\n{"="*35}\n', *genders)

# Commit and close connection
connection.commit()
connection.close()

Number of Employees by Gender
 Females: 19
 Males: 18



## Provide a count of the employees by dept name

In [7]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Select data
cursor.execute('''
    select departments.description,count(dept_code) from employees
    left join departments on employees.dept_code=departments.code
    group by dept_code
    ''',
    )
# Fetch query
depts = [f'{dept[0]}: {dept[1]}\n' for dept in cursor.fetchall()]
print(f'Number of Employees by Department:\n{"="*35}\n', *depts)

# Commit and close connection
connection.commit()
connection.close()

Number of Employees by Department:
 Accounting: 7
 Executive: 1
 Finance: 7
 Human Resources: 6
 Information Technology: 9
 Warehouse: 7



## Provide a list of employees in last name, first name order

In [8]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Select data
cursor.execute('''
    select last_name||\', \'||first_name as name from employees order by name
    '''
)
# Fetch query
names = [f'{name[0]}\n' for name in cursor.fetchall()]
print(f'Employee Names by Last Name:\n{"="*35}\n', *names)

# Commit and close connection
connection.commit()
connection.close()

Employee Names by Last Name:
 Breen, Margret
 Davidson, Fred
 Davidson, Wilma
 Davis, Elizabeth
 Ellis, Francis
 Forrest, Peter
 Freach, Greta
 French, Sara
 Gantt, Jackson
 Holland, Henry
 Jackson, Jessica
 Jarvis, Melvin
 Jones, Jessica
 Jones, Pamala
 Keller, Bradley
 Lagos, Leeland
 Lee, Tatum
 Martinez, Margarita
 Nulland, Nicholas
 Opus, Oliver
 Pollard, Franz
 Quinones, Maribel
 Ramos, Diego
 Regis, Lucy
 Rivera, Jewels
 Sanchez, Jorge
 Saulzman, Randle
 Saunders, Asia
 Sing, Hu
 Sung, Henry
 Tate, Rachael
 Torrez, Lea
 Ute, Ulsula
 Williams, William
 Williamson, Ursula
 Xi, Yu
 Yee, Sue



## Provide a list of employees in tenure order (active employees only) in descending order

In [9]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Select data
cursor.execute('''
    select employees.last_name||\', \'||employees.first_name as name,
        service_dates.years_of_service from employees
    join service_dates on employees.employee_id=service_dates.employee_id
    where service_dates.term_date=''
    order by hire_date asc
    '''
)
# Fetch query
years_of_service = [f'{years[0]} (Years of Service: {years[1]})\n' for years in cursor.fetchall()]
print(f'Employee Names by Years of Service:\n{"="*35}\n', *years_of_service)

# Commit and close connection
connection.commit()
connection.close()

Employee Names by Years of Service:
 Saulzman, Randle (Years of Service: 20)
 Davidson, Fred (Years of Service: 18)
 Jones, Pamala (Years of Service: 18)
 Ellis, Francis (Years of Service: 16)
 Pollard, Franz (Years of Service: 14)
 Williamson, Ursula (Years of Service: 13)
 Ute, Ulsula (Years of Service: 13)
 Freach, Greta (Years of Service: 12)
 Forrest, Peter (Years of Service: 11)
 Nulland, Nicholas (Years of Service: 11)
 Holland, Henry (Years of Service: 10)
 Keller, Bradley (Years of Service: 10)
 Williams, William (Years of Service: 10)
 Jones, Jessica (Years of Service: 8)
 Rivera, Jewels (Years of Service: 8)
 Regis, Lucy (Years of Service: 7)
 Opus, Oliver (Years of Service: 7)
 Sung, Henry (Years of Service: 7)
 Quinones, Maribel (Years of Service: 6)
 Gantt, Jackson (Years of Service: 6)
 Ramos, Diego (Years of Service: 6)
 French, Sara (Years of Service: 6)
 Saunders, Asia (Years of Service: 5)
 Tate, Rachael (Years of Service: 5)
 Davis, Elizabeth (Years of Service: 5)
 

## Provide a list of employees by manager name in alphabetical order by last name

In [10]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Select data
cursor.execute('''
    select a.employee_id, a.first_name||\', \'||a.last_name as manager from employees a, employees b
    where a.employee_id=b.manager_id
    group by a.last_name
    '''
)
manager_ids = [name for name in cursor.fetchall()]
for manager_id in manager_ids:
    cursor.execute('''
        select last_name||\', \'||first_name as employee
        from employees
        where manager_id=:manager_id
        ''',
        {'manager_id': manager_id[0]}
    )
# Fetch query
    emps = [f'{name[0]}\n' for name in cursor.fetchall()]
    print(f'Manager: {manager_id[1]}\n{"="*35}\n', *emps)

# Commit and close connection
connection.commit()
connection.close()

Manager: Fred, Davidson
 Williams, William
 Davis, Elizabeth
 Jarvis, Melvin
 Quinones, Maribel
 Nulland, Nicholas
 Saunders, Asia

Manager: Francis, Ellis
 Yee, Sue
 Tate, Rachael
 Williamson, Ursula
 French, Sara
 Ute, Ulsula

Manager: Peter, Forrest
 Sing, Hu
 Breen, Margret
 Opus, Oliver
 Lee, Tatum
 Jackson, Jessica
 Jones, Jessica

Manager: Pamala, Jones
 Ramos, Diego
 Sanchez, Jorge
 Sung, Henry
 Pollard, Franz
 Keller, Bradley
 Lagos, Leeland
 Holland, Henry
 Torrez, Lea

Manager: Randle, Saulzman
 Ellis, Francis
 Jones, Pamala
 Davidson, Fred
 Davidson, Wilma
 Xi, Yu
 Forrest, Peter
 Saulzman, Randle

Manager: Yu, Xi
 Rivera, Jewels
 Freach, Greta
 Gantt, Jackson
 Martinez, Margarita
 Regis, Lucy



## Provide a list of employees by region

In [11]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

regions = [
    'NORTH',
    'NORTH EAST',
    'NORTH WEST',
    'SOUTH',
    'SOUTH EAST',
    'SOUTH WEST',
    'EAST',
    'WEST'
]

# Select data
for region in regions:
    cursor.execute('''
        select employees.last_name||', '||employees.first_name as name,regions.description as region
        from employees join regions on employees.region_code=regions.code
        where regions.description=:region
        ''',
        {'region': region}
    )
    # Fetch query
    query = [f'{row[0]}\n' for row in cursor.fetchall()]
    print(f'{region}:\n{"="*35}\n', *query) if len(query) > 1 else False

# Commit and close connection
connection.commit()
connection.close()

NORTH EAST:
 Davidson, Fred
 Ellis, Francis
 Jones, Pamala
 Ramos, Diego
 Rivera, Jewels
 Sanchez, Jorge
 Sing, Hu
 Yee, Sue

NORTH WEST:
 Jackson, Jessica
 Lee, Tatum
 Saunders, Asia
 Ute, Ulsula

SOUTH EAST:
 Forrest, Peter
 Holland, Henry
 Jones, Jessica
 Lagos, Leeland
 Martinez, Margarita
 Regis, Lucy
 Torrez, Lea

SOUTH WEST:
 Davidson, Wilma
 Davis, Elizabeth
 Freach, Greta
 Gantt, Jackson
 Jarvis, Melvin
 Sung, Henry
 Tate, Rachael
 Williams, William

EAST:
 Breen, Margret
 Keller, Bradley
 Nulland, Nicholas
 Opus, Oliver
 Pollard, Franz
 Saulzman, Randle

WEST:
 French, Sara
 Quinones, Maribel
 Williamson, Ursula
 Xi, Yu



## Provide a list of employees who have exceeded their allotment of sick days

In [12]:
# Connect to database
connection = sqlite3.connect('employee_data.db')
cursor = connection.cursor()

# Select data
cursor.execute('''
    select last_name||', '||first_name as name,available_sick_days from employees
    where available_sick_days<0
    '''
)
# Fetch query
sick_days = [f'{day[0]} (Available Sick Days: {day[1]})\n' for day in cursor.fetchall()]
print(f'Employees who have exceeded their alloted sick days:\n{"="*52}\n', *sick_days)

# Commit and close connection
connection.commit()
connection.close()

Employees who have exceeded their alloted sick days:
 Sing, Hu (Available Sick Days: -2)
 Ramos, Diego (Available Sick Days: -2)
 Sanchez, Jorge (Available Sick Days: -4)
 Yee, Sue (Available Sick Days: -2)
 Williamson, Ursula (Available Sick Days: -2)
 French, Sara (Available Sick Days: -2)
 Opus, Oliver (Available Sick Days: -2)
 Ute, Ulsula (Available Sick Days: -2)
 Jackson, Jessica (Available Sick Days: -4)
 Lagos, Leeland (Available Sick Days: -2)
 Martinez, Margarita (Available Sick Days: -2)
 Jones, Jessica (Available Sick Days: -2)

